In [3]:
import pathlib
import torch
import torchvision
import os
from PIL import Image
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display # to display images
import sys
sys.path.append("../")

from helper_code.resnet_model import ResnetModel
from helper_code.mario_buttons_dataset import TEST_WORLDS, MarioEpisode,MarioButtonsDataset


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.manual_seed(17)


batch_size = 128
learning_rate = 1e-3
epochs = 7
    
group =3#3
use_color = False

preload=False


print(f"loading dataset preload:{preload}")

transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((256,256)),
        torchvision.transforms.ToTensor(),
        
        #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

#mario_epsode = MarioEpisode('../mario_dataset/Rafael_hwfrb471_e3_8-3_win/',group_frames=group,use_color=use_color,transform=transforms,preload=False)
mario_dataset = MarioButtonsDataset('C:\\Users\\lavi\Desktop\\deep_learning\\Mario_IDM_DL\\mario_dataset\\'
                                    ,group_frames=group,use_color=use_color,worlds=TEST_WORLDS,preload=False)
def _extract_action(file_name):
        #print(pathlib.PurePath(file_name).name)
        action_number = pathlib.PurePath(file_name).name.split("_")[5][1:]
        action_tensor = torch.zeros(8)
        action_bin = str(bin(int(action_number)))[2:].zfill(8) #remove 0b in start
        #print(action_bin)
        for i in range(8):
            action_tensor[i] = int(action_bin[i])
        return action_tensor

def calc_acc():
    tot_frames = 0
    #mean action :tensor([0.2000, 0.0000, 0.0900, 0.8100, 0.0000, 0.5400, 0.0000, 0.0000])
    mean = torch.Tensor([0.2000, -1, 0.0900, 0.8100, -1, 0.5400, -1,-1])
    button_hist = {}
    guess = torch.round(mean)
    tot_correct = 0
    for ep in tqdm(os.listdir('../mario_dataset/')):
        if not os.path.isdir(os.path.join('../mario_dataset/',ep)):
            continue
        for frame in os.listdir(f'../mario_dataset/{ep}/'):
            tot_frames += 1 
            action = _extract_action(frame)
            tot_correct += (guess == action).sum().item()
            for i in range(8):
                if f"{i}-{action[i]}" not in button_hist:
                    button_hist[f"{i}-{action[i]}"] = 0
                button_hist[f"{i}-{action[i]}"] += 1

    print(f"accuracy:{tot_correct/(tot_frames*4)}")
    print(button_hist)
    plt.figure(figsize=(20,10))
    plt.bar(button_hist.keys(), button_hist.values(), 1.0, color='g')
    plt.show()

def calc_word_level_hist():
    word_hist = {}
    for ep in tqdm(os.listdir('../mario_dataset/')):
        if not os.path.isdir(os.path.join('../mario_dataset/',ep)):
            continue
        for frame in os.listdir(f'../mario_dataset/{ep}/'):
            metadata = pathlib.PurePath(frame).name.split("_")
            world = metadata[3].split("-")[0]
            level = metadata[3].split("-")[1]
            if f"{world}-{level}" not in word_hist:
                word_hist[f"{world}-{level}"] = 0
            word_hist[f"{world}-{level}"] += 1
    print(word_hist)
    plt.figure(figsize=(20,10))
    plt.bar(word_hist.keys(), word_hist.values(), 1.0, color='g')
    plt.show()

#calc_acc()
# tot_action = torch.zeros(8,dtype=torch.float32)
# for ep in tqdm(os.listdir('../mario_dataset/')):
#     if not os.path.isdir(os.path.join('../mario_dataset/',ep)):
#          continue
#     for frame in os.listdir(f'../mario_dataset/{ep}/'):
#         tot_frames += 1
#         tot_action += _extract_action(frame)
        
# tot_action = torch.round(tot_action/tot_frames,decimals=2)
# print(f"mean action :{tot_action}")


# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# model = ResnetModel(group_size=group,use_color=use_color,use_pretrained=False).to(device)
# model.load_state_dict(torch.load('C:\\Users\\lavi\Desktop\\deep_learning\\Mario_IDM_DL\\models\\checkpoint_1_2500.pt',map_location=device))

# model.eval()
# with torch.no_grad():
#     for i in [0,10,100,1000]:
#         img,action = mario_dataset[0]
#         img = img.unsqueeze(0)
#         img = img.to(device)
#         pred = torch.sigmoid(model(img))
#         print(f"pred:{pred.cpu().numpy()},action:{action}")

cuda:0
loading dataset preload:False
total episodes:53


In [4]:
word_hist = {}
for ep in tqdm(os.listdir('../mario_dataset/')):
    if not os.path.isdir(os.path.join('../mario_dataset/',ep)):
        continue
    for frame in os.listdir(f'../mario_dataset/{ep}/'):
        metadata = pathlib.PurePath(frame).name.split("_")
        world = metadata[3].split("-")[0]
        level = metadata[3].split("-")[1]
        if f"{world}-{level}" not in word_hist:
            word_hist[f"{world}-{level}"] = 0
        word_hist[f"{world}-{level}"] += 1
print(word_hist)
#plt.figure(figsize=(20,10))
#plt.bar(word_hist.keys(), word_hist.values(), 1.0, color='g')
#plt.show()

100%|██████████| 281/281 [00:05<00:00, 48.05it/s]

{'8-1': 123306, '8-2': 143530, '8-3': 79401, '8-4': 45605, '1-1': 10811, '2-1': 15654, '2-2': 16003, '2-3': 12177, '2-4': 7319, '3-1': 14782, '3-2': 8170, '3-3': 7016, '3-4': 5057, '4-1': 13178, '1-2': 16209, '4-2': 14493, '4-3': 9569, '4-4': 4621, '5-1': 16688, '5-2': 19412, '5-3': 8925, '1-3': 9262, '1-4': 4870, '5-4': 4134, '6-1': 17129, '6-2': 22993, '6-3': 12102, '6-4': 8188, '7-1': 24437, '7-2': 16580, '7-3': 11934, '7-4': 13579}
